***"""
<br>@Project: LTRO Project - Team 4 - Tool for Forecasting,Clustering and VRP
<br>@author: Harihareshwar Kumaravel
<br>"""***

**Script to forecast at a particular point for a type of waste**

In [ ]:
import pandas as pd 
import numpy as np
from statsmodels.tsa.arima.model import ARIMA
import statsmodels.tsa.statespace.sarimax as sarimax
from statsmodels.tsa.holtwinters import ExponentialSmoothing
from pmdarima.arima import auto_arima
from sklearn.metrics import mean_squared_error
from datetime import timedelta
import warnings
warnings.filterwarnings("ignore")

# ///INPUTS///


p = 'BX'
q = 'Verre'
r = "OM"
s = "Carton"
c = []
loc = "datas/Data cleared.xlsx"  # Location where the cleaned data is present
'''d = '191' #Enter the point where the prediction has to be made. Make sure it is in 'str' format
m = 1 # m = 0,1,2,3 for BX,Verre,OM,Carton respectively'''

# ///FUNCTIONS///

def SelectPoints(point, m):
    df = pd.read_excel(loc , sheet_name= m)
    df = df[df['date'] >= '01-01-2018']
    df['n_point']= df['n_point'].astype(str)
    df = df[df["n_point"] == point]
    df['taux'] = df['taux'].fillna(value = np.mean(df['taux']))
    df = df.query('taux <= 1.0')
    df['date']=pd.to_datetime(df['date'])
    df['day'] = [i.day for i in df['date']]
    df['month'] = [i.month for i in df['date']]
    df['year'] = [i.year for i in df['date']]
    df['week'] = [i.week for i in df['date']]
    if m == 0:
        for i in range(10):
            c.append(p)
    elif m == 1:
        for i in range(10):
            c.append(q) 
    elif m == 1:
        for i in range(10):
            c.append(r) 
    else:
        for i in range(10):
            c.append(s)
    return df

#n = 0,1,2,3 for BX,Verre,OM,Carton respectively
#Select the point to be forecasted and the type of waste. 
'''df = SelectPoints(d,m)  
df['days before last pick up'] = df['days before last pick up'].apply(pd.to_numeric, errors='coerce')
n = df['days before last pick up'].mean(skipna=True)'''

#Spliting the data to evaluate the models using MSE(Mean Squared error)
def test_train_split(df):
    train_df = df['taux'].iloc[:round(len(df)*0.8)]
    test_df = df['taux'].iloc[round(len(df)*0.8):]
    train_df = pd.DataFrame(train_df)
    test_df = pd.DataFrame(test_df)
    return train_df,test_df

#Functions to evaluate various Time series forecasting models

def esEval(df):  #exponential smoothing Evaluation 
    test_train_split(df)
    train_df = test_train_split(df)[0]
    test_df = test_train_split(df)[1]
    es_model = ExponentialSmoothing(train_df['taux'])
    es_model_fit = es_model.fit()
    # Make predictions on the test set
    es_predictions = es_model_fit.predict(start=len(train_df), end=len(train_df)+len(test_df)-1)
    #es_predictions = es_model_fit.forecast(len(train_df)+len(test_df)-1)
    es_mse =  mean_squared_error(test_df['taux'], es_predictions)
    es_mse = es_mse**0.5
    return es_mse

def arEval(df): #AutoRegressive Model
    test_train_split(df)
    train_df = test_train_split(df)[0]
    test_df = test_train_split(df)[1]
    ar_model = ARIMA(train_df["taux"], order=(1,0,0))
    ar_model_fit = ar_model.fit()
    ar_predictions = ar_model_fit.predict(start=len(train_df), end=len(train_df)+len(test_df)-1, dynamic=False)
    ar = pd.DataFrame(ar_predictions)
    ar_mse = mean_squared_error(test_df['taux'],ar)
    ar_mse = ar_mse**0.5
    return ar_mse

def armaEval(df): #AutoRegressive Moving average model
    test_train_split(df)
    train_df = test_train_split(df)[0]
    test_df = test_train_split(df)[1]
    arma_model = ARIMA(train_df["taux"], order=(1,0,1))
    arma_model_fit = arma_model.fit()
    arma_predictions = arma_model_fit.predict(start=len(train_df), end=len(train_df)+len(test_df)-1, dynamic=False)
    arma = pd.DataFrame(arma_predictions)
    arma_mse = mean_squared_error(test_df['taux'],arma)
    arma_mse = arma_mse**0.5
    return arma_mse

def arimaEval(df): #AutoRegressive Integrated Moving Average
    test_train_split(df)
    train_df = test_train_split(df)[0]
    test_df = test_train_split(df)[1]
    arima_model = auto_arima(df['taux'],start_p=1, max_P=1, start_q=1, max_Q=1, max_d=1, max_D=1, m=7,trace=False)
    a =str(arima_model.df_model)
    b = order_extract(a)
    arima_model = ARIMA(train_df["taux"], order=b[0])
    arima_model_fit = arima_model.fit()
    arima_predictions = arima_model_fit.predict(start=len(train_df), end=len(train_df)+len(test_df)-1, dynamic=False)
    arima = pd.DataFrame(arima_predictions)
    arima_mse = mean_squared_error(test_df['taux'],arima)
    arima_mse = arima_mse**0.5
    return arima_mse

def order_extract(a : str):
    string = a
    order_start = string.index('order')
    order_start_paren = string.index('(', order_start)
    order_end_paren = string.index(')', order_start_paren)
    order_string = string[order_start_paren+1:order_end_paren]
    order = tuple(map(int, order_string.split(',')))
    seasonal_order_start = string.index('seasonal_order')
    seasonal_order_start_paren = string.index('(', seasonal_order_start)
    seasonal_order_end_paren = string.index(')', seasonal_order_start_paren)
    seasonal_order_string = string[seasonal_order_start_paren+1:seasonal_order_end_paren]
    seasonal_order = tuple(map(int, seasonal_order_string.split(',')))
    return order, seasonal_order

def sarimaEval(df): #Seasonal Auto Regressive Integrated Moving Average
    test_train_split(df)
    train_df = test_train_split(df)[0]
    test_df = test_train_split(df)[1]
    arima_model = auto_arima(df['taux'],start_p=1, max_P=1, start_q=1, max_Q=1, max_d=1, max_D=1, m=7,trace=False)
    a =str(arima_model.df_model)
    global b 
    b =  order_extract(a)
    model = sarimax.SARIMAX(train_df["taux"], order=b[0], seasonal_order=b[1])  #Results from "arima_model.summary()". Look at Model and fill it in here
    model_fit = model.fit()
    predictions = model_fit.predict(start=len(train_df), end=len(train_df)+len(test_df)-1)
    sarima = pd.DataFrame(predictions)
    sarima_mse = mean_squared_error(test_df['taux'],sarima)
    sarima_mse = sarima_mse**0.5
    return sarima_mse

# Methods to forecast data

def esModel(df):  #exponential Smoothing model
    series =  df['taux'].to_list()
    alpha = 0.9
    n_pred = 10
    forecast = [series[0]] # initial forecast
    for i in range(1, len(series)):
        forecast.append((alpha * series[i] + (1 - alpha) * forecast[i-1]))
    for i in range(n_pred):
        forecast.append((alpha * forecast[-1] + (1 - alpha) * forecast[-1])) 
    pred = forecast[-10:]
    df1 = pd.DataFrame({'prediction': []})
    df2 = pd.DataFrame()
    a = df['n_point'].unique()[0]
    b = []
    df1['prediction'] = pred
    timestamp = df['date'].iloc[-1] + timedelta(days = n)
    for i in range(len(df1)):
        df2 = df2.append({'date':timestamp},ignore_index=True)
        timestamp = df2['date'].iloc[i] + timedelta(days=n)
    pred_df = pd.concat([df1, df2], axis=0, ignore_index=True)
    for i in range(len(df1)):
        pred_df['date'][i] = pred_df['date'][i+len(df1)]
    pred_df = pred_df.dropna()
    for i in range(len(df1)):
        b.append(a)
    pred_df.insert(2, 'n_point', b)
    #pred_df.insert(3, 'waste_type', c)
    es = pred_df
    return es

def arModel(df):  #AutoRegressive Model
    model = ARIMA(df["taux"], order=(1,0,0))
    model_fit = model.fit()
    predictions = model_fit.predict(start=len(df), end=len(df)+9, dynamic=False)
    df1 = pd.DataFrame({'prediction': []})
    df2 = pd.DataFrame()
    a = df['n_point'].unique()[0]
    b = []
    df1['prediction'] = predictions
    timestamp = df['date'].iloc[-1] + timedelta(days = n)
    for i in range(len(df1)):
        df2 = df2.append({'date':timestamp},ignore_index=True)
        timestamp = df2['date'].iloc[i] + timedelta(days=n)
    pred_df = pd.concat([df1, df2], axis=0, ignore_index=True)
    for i in range(len(df1)):
        pred_df['date'][i] = pred_df['date'][i+len(df1)]
    pred_df = pred_df.dropna()
    for i in range(len(df1)):
        b.append(a)
    pred_df.insert(2, 'n_point', b)
    #pred_df.insert(3, 'waste_type', c)
    ar = pred_df
    return ar

def armaModel(df): #ARMA model
    model = ARIMA(df["taux"], order=(1,0,1))
    model_fit = model.fit()
    predictions = model_fit.predict(start=len(df), end=len(df)+9, dynamic=False)
    df1 = pd.DataFrame({'prediction': []})
    df2 = pd.DataFrame()
    a = df['n_point'].unique()[0]
    b = []
    df1['prediction'] = predictions
    timestamp = df['date'].iloc[-1] + timedelta(days = n)
    for i in range(len(df1)):
        df2 = df2.append({'date':timestamp},ignore_index=True)
        timestamp = df2['date'].iloc[i] + timedelta(days=n)
    pred_df = pd.concat([df1, df2], axis=0, ignore_index=True)
    for i in range(len(df1)):
        pred_df['date'][i] = pred_df['date'][i+len(df1)]
    pred_df = pred_df.dropna()
    for i in range(len(df1)):
        b.append(a)
    pred_df.insert(2, 'n_point', b)
    #pred_df.insert(3, 'waste_type', c)
    arma = pred_df
    return arma

def arimaModel(df):  #ARIMA Model
    arima_model = auto_arima(df['taux'],m=7,trace=False)
    a =str(arima_model.df_model)
    b = order_extract(a)
    model = ARIMA(df["taux"], order=b[0])
    model_fit = model.fit()
    predictions = model_fit.predict(start=len(df), end=len(df)+9, dynamic=False)
    df1 = pd.DataFrame({'prediction': []})
    df2 = pd.DataFrame()
    a = df['n_point'].unique()[0]
    b = []
    # Make predictions using the fitted model
    df1['prediction'] = predictions
    timestamp = df['date'].iloc[-1] + timedelta(days = n)
    for i in range(len(df1)):
        df2 = df2.append({'date':timestamp},ignore_index=True)
        timestamp = df2['date'].iloc[i] + timedelta(days=n)
    pred_df = pd.concat([df1, df2], axis=0, ignore_index=True)
    for i in range(len(df1)):
        pred_df['date'][i] = pred_df['date'][i+len(df1)]
    pred_df = pred_df.dropna()
    for i in range(len(df1)):
        b.append(a)
    pred_df.insert(2, 'n_point', b)
    #pred_df.insert(3, 'waste_type', c)
    arima = pred_df
    return arima

def sarimaModel(df):  #SARIMA Model
    arima_model = auto_arima(df['taux'],m=7,trace=False)
    a =str(arima_model.df_model)
    b = order_extract(a)
    model = sarimax.SARIMAX(df["taux"], order=b[0], seasonal_order=b[1])
    model_fit = model.fit()
    predictions = model_fit.predict(start=len(df), end=len(df)+9, dynamic=False)
    df1 = pd.DataFrame({'prediction': []})
    df2 = pd.DataFrame()
    a = df['n_point'].unique()[0]
    b = []
    # Make predictions using the fitted model
    df1['prediction'] = predictions
    timestamp = df['date'].iloc[-1] + timedelta(days = n)
    for i in range(len(df1)):
        df2 = df2.append({'date':timestamp},ignore_index=True)
        timestamp = df2['date'].iloc[i] + timedelta(days=n)
    pred_df = pd.concat([df1, df2], axis=0, ignore_index=True)
    for i in range(len(df1)):
        pred_df['date'][i] = pred_df['date'][i+len(df1)]
    pred_df = pred_df.dropna()
    for i in range(len(df1)):
        b.append(a)
    pred_df.insert(2, 'n_point', b)
    #pred_df.insert(3, 'waste_type', c)
    sarima = pred_df
    return sarima

def prediction(df):  # Use the prediction function of the model with the lowest MSE to make predictions
    min_rmse = min(esEval(df),arEval(df),armaEval(df),arimaEval(df),sarimaEval(df))
    print('RMSE using Exponential smoothing: ',esEval(df))
    print('RMSE using AR Model: ', arEval(df))
    print('RMSE using ARMA Model: ', armaEval(df))
    print('RMSE using ARIMA Model: ', arimaEval(df))
    print('RMSE using SARIMA Model: ', sarimaEval(df))
    if min_rmse == esEval(df):
        print('minimun RMSE was found using Exponential Smoothing ')
        print('min_mse =',min_rmse)
        return esModel(df)
    elif min_rmse == arEval(df):
        print('minimun RMSE was found using AR Model ')
        print('min_mse =',min_rmse)
        return arModel(df)
    elif min_rmse == armaEval(df):
        print('minimun RMSE was found using ARMA Model ')
        print('min_mse =',min_rmse)
        return armaModel(df)
    elif min_rmse == arimaEval(df):
        print('minimun RMSE was found using ARIMA Model ')
        print('min_mse =',min_rmse)
        return arimaModel(df)
    else:
        print('minimun RMSE was found using SARIMA Model ')
        print('min_mse =',min_rmse)
        return sarimaModel(df)
        
def minRMSE(df):
    min_rmse = min(esEval(df),arEval(df),armaEval(df),arimaEval(df),sarimaEval(df))
    return min_rmse

***User Inputs***

In [ ]:
d = '99-1' #Enter the point where the prediction has to be made. Make sure it is in 'str' format
m = 0 # m = 0,1,2,3 for BX,Verre,OM,Carton respectively

df = SelectPoints(d,m)  
df['days before last pick up'] = df['days before last pick up'].apply(pd.to_numeric, errors='coerce')
n = df['days before last pick up'].mode()
n = round(n[0])
prediction(df)

"Predictions.xlsx" contains the predicted values for different types of wastes(inside the datas folder)

**Clustering**

In [ ]:
# ///LIBRAIRIES///

from sklearn.cluster import KMeans
from yellowbrick.cluster import KElbowVisualizer
import pandas as pd
import numpy as np

# ///USER INPUTS///
date = '2022-07-01' #Date on which the clustering need to be done
m = 0 # Input type of waste - 0,1,2,3 for BX,Verre,OM,Carton respectively

# ///OTHER VARIABLES///
dist = pd.read_excel("Distances.xlsx", sheet_name=1) #Distance from the depot to various location

# ///FUNCTIONS///

def clean_column(value):  #To clean the column to get int datatype
    if isinstance(value, int):
        return value
    elif isinstance(value, str):
        try:
            return int(value)
        except ValueError:
            return value

def Selectpoints(date):   #Selects datapoints from the predicted data for a single date
    date = date
    df = pd.read_excel("Predictions.xlsx", sheet_name = m )
    df['date']=pd.to_datetime(df['date'])
    df['n_point'] = df['n_point'].astype(str)
    df['day'] = [i.day for i in df['date']]
    df['month'] = [i.month for i in df['date']]
    df['year'] = [i.year for i in df['date']]
    df['week'] = [i.week for i in df['date']]
    df = df.loc[df['date'] == date]
    df[['points', 'bin_no']] = df['n_point'].str.extract(r'(\d+)-?(\d*)')
    df['points'] = df['points'].fillna(value=df['n_point'])
    df = df.loc[:,('date','n_point','prediction','points','bin_no')]
    #df = df.query('prediction >= 0.7')
    df= df.reset_index()
    df['points'] = df['points'].apply(clean_column)
    df['dist'] = 0
    for i in range (len(df)):
        for j in range(len(dist)):
            if df['points'][i] == dist['n_point'][j]:
                df.loc[i, 'dist'] = dist.loc[j,'dist']
    return df

def OptimumClusters(df):
    X = df[['dist','prediction']]

    km = KMeans()
    visualizer = KElbowVisualizer(km, k=(2,8)) #Number of available trucks is 7
    
    visualizer.fit(X)        # Fit the data to the visualizer
    visualizer.show()        # Finalize and render the figure
    return visualizer.elbow_value_

def Clustering(df):  #Clusters the data based on prediction and distance from depot using k-means algorithm 
    X = df[['dist','prediction']]
    km = KMeans(n_clusters = OptimumClusters(df))
    y_predicted = km.fit_predict(X)

    df['cluster'] = y_predicted
    b = df['cluster'].unique()
    a =[]
    for i in range(len(b)):
        c = df[df.cluster == i]
        a.append(c['points'].unique())
    data = []
    for arr in a:
        data.append(arr.tolist())
    data = [[elem-1 for elem in sublist] for sublist in data]
    return data

df = Selectpoints(date)

print(Clustering(df))


Copy the above result and paste in 'loc' variable in the below chunk

**Vehicle Routing Problem** <br> This here is treated as a TSP to get the optimum route of the clusters obtained previously. The routing may or maynot be optimal

In [ ]:
from ortools.constraint_solver import routing_enums_pb2
from ortools.constraint_solver import pywrapcp
import pandas as pd

#Enter the list of locations where VRP needs to be implemented

#loc is the output from clustering
loc = [[77, 197, 104, 159, 147, 151, 213, 146, 226, 227, 228, 165, 204], [4, 264, 61], [102, 90, 100, 98, 125, 91, 97, 241, 223, 121], [80, 160, 272]] #Paste the result from above chunk here to execute the VRP
global b
b = []

def distMat(loc):
    loc1 = loc.copy()
    loc1 = np.insert(loc,0,328) # depot location
    df = pd.read_excel("D:/Grenoble INP-SIE/Semester 3/Logistics and OR/PROJECTS/PROJECTS/LTRO_Project Files/Project Codes/Distances.xlsx", sheet_name=2, header=None)
    df = df.loc[loc1,loc1] # to extract distance matrix for specific locations alone
    m = df.to_numpy() # to create a numpy array for distance matrix
    return m

def create_data_model(dmat):
    """Stores the data for the problem."""
    data = {}
    data['distance_matrix'] = dmat
    data['num_vehicles'] = 1
    data['depot'] = 0
    return data

def print_solution(manager, routing, solution):
    """Prints solution on console."""
    ###print('Objective: {} metres'.format(solution.ObjectiveValue()))
    index = routing.Start(0)
    plan_output = '\n'
    route_distance = 0
    global a
    a = []
    while not routing.IsEnd(index):
        #plan_output += ' {} , '.format(manager.IndexToNode(index))
        a.append(manager.IndexToNode(index))
        previous_index = index
        index = solution.Value(routing.NextVar(index))
        route_distance += routing.GetArcCostForVehicle(previous_index, index, 0)
    a.pop(0)
    #plan_output += ' {}\n'.format(manager.IndexToNode(index))
    plan_output += 'Route distance: {}metres'.format(route_distance)
    print(plan_output)
    return a

def vrp(loc):
    pl_list = []
    for i in range(len(loc)):
        pl = loc[i]
        pl_list.append(distMat(pl))
    
    for i in range(len(loc)):
        """Entry point of the program."""
        # Instantiate the data problem.
        data = create_data_model(pl_list[i])

        # Create the routing index manager.
        manager = pywrapcp.RoutingIndexManager(len(data['distance_matrix']),
                                            data['num_vehicles'], data['depot'])

        # Create Routing Model.
        routing = pywrapcp.RoutingModel(manager)


        def distance_callback(from_index, to_index):
            """Returns the distance between the two nodes."""
            # Convert from routing variable Index to distance matrix NodeIndex.
            from_node = manager.IndexToNode(from_index)
            to_node = manager.IndexToNode(to_index)
            return data['distance_matrix'][from_node][to_node]

        transit_callback_index = routing.RegisterTransitCallback(distance_callback)

        # Define cost of each arc.
        routing.SetArcCostEvaluatorOfAllVehicles(transit_callback_index)

        # Setting first solution heuristic.
        search_parameters = pywrapcp.DefaultRoutingSearchParameters()
        search_parameters.first_solution_strategy = (
            routing_enums_pb2.FirstSolutionStrategy.PATH_CHEAPEST_ARC)

        # Solve the problem.
        solution = routing.SolveWithParameters(search_parameters)

        # Print solution on console.
        if solution:
            print_solution(manager, routing, solution)

        values = loc[i]
        order = a
        Order_visit = sort_list(values, order)
        Order_visit.append(328)
        Order_visit.insert(0,328)
        Visit_Order(Order_visit)
        print('Tour' + ' ' + str(i+1)+ ':' , Order_visit) 
    print ('\n',b)

def Visit_Order(Order_visit):
    b.append(Order_visit)
    
def sort_list(values, order):
    # Zip the values and order lists together
    zipped_lists = zip(values, order)

    # Sort the zipped list by the order of the elements in the second list
    sorted_zipped_lists = sorted(zipped_lists, key=lambda x: x[1])

    # Unzip the sorted list and return the first list (the values list)
    sorted_values, _ = zip(*sorted_zipped_lists)

    sorted_values = np.array(sorted_values)
    sorted_values = sorted_values.tolist()
    return sorted_values

vrp(loc)

The Final line is the result of VRP which can be used to visualize the route on that particular date